In [1]:
import paddle
from paddle import nn

In [2]:
L = nn.Linear(256, 2048)
L2 = nn.Linear(2048, 256)

/workspace/DeepSpeech-2.x/tools/venv-dev/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [3]:
import numpy as np
import torch


In [4]:
x = np.random.randn(2, 51, 256)
print(x.dtype)
px = paddle.to_tensor(x, dtype='float32')
tx = torch.tensor(x, dtype=torch.float32)
print(px)
print(tx)

float64
Tensor(shape=[2, 51, 256], dtype=float32, place=CUDAPlace(0), stop_gradient=True,
       [[[-1.54171216, -2.61531472, -1.79881978, ..., -0.31395876,  0.56513089, -0.44516513],
         [-0.79492962,  1.91157901,  0.66567147, ...,  0.54825783, -1.01471853, -0.84924090],
         [-1.22556651, -0.36225814,  0.65063190, ...,  0.65726501,  0.05563191,  0.09009409],
         ...,
         [ 0.38615900, -0.77905393,  0.99732304, ..., -1.38463700, -3.32365036, -1.31089687],
         [ 0.05579993,  0.06885809, -1.66662002, ..., -0.23346378, -3.29372883,  1.30561364],
         [ 1.90676069,  1.95093191, -0.28849599, ..., -0.06860496,  0.95347673,  1.00475824]],

        [[-0.91453546,  0.55298805, -1.06146812, ..., -0.86378336,  1.00454640,  1.26062179],
         [ 0.10223761,  0.81301165,  2.36865163, ...,  0.16821407,  0.29240361,  1.05408621],
         [-1.33196676,  1.94433689,  0.01934209, ...,  0.48036841,  0.51585966,  1.22893548],
         ...,
         [-0.19558455, -0.47075930

/workspace/DeepSpeech-2.x/tools/venv-dev/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [5]:
data = np.load('enc_0_ff_out.npz', allow_pickle=True)
t_norm_ff = data['norm_ff']
t_ff_out = data['ff_out']
t_ff_l_x = data['ff_l_x']
t_ff_l_a_x = data['ff_l_a_x']
t_ff_l_a_l_x = data['ff_l_a_l_x']
t_ps = data['ps']

In [6]:
L.set_state_dict({'weight': t_ps[0].T, 'bias': t_ps[1]})
L2.set_state_dict({'weight': t_ps[2].T, 'bias': t_ps[3]})

ps = []
for n, p in L.named_parameters():
   ps.append(p)

for n, p in L2.state_dict().items():
    ps.append(p)
    
for p, tp in zip(ps, t_ps):
    print(np.allclose(p.numpy(), tp.T))

True
True
True
True


In [7]:
# data = np.load('enc_0_ff_out.npz', allow_pickle=True)
# t_norm_ff = data['norm_ff']
# t_ff_out = data['ff_out']
# t_ff_l_x = data['ff_l_x']
# t_ff_l_a_x = data['ff_l_a_x']
# t_ff_l_a_l_x = data['ff_l_a_l_x']
# t_ps = data['ps']
TL = torch.nn.Linear(256, 2048)
TL2 = torch.nn.Linear(2048, 256)
TL.load_state_dict({'weight': torch.tensor(t_ps[0]), 'bias': torch.tensor(t_ps[1])})
TL2.load_state_dict({'weight': torch.tensor(t_ps[2]), 'bias': torch.tensor(t_ps[3])})

# for n, p in TL.named_parameters():
#    print(n, p)
# for n, p in TL2.named_parameters():
#    print(n, p)

ps = []
for n, p in TL.state_dict().items():
    ps.append(p.data.numpy())
    
for n, p in TL2.state_dict().items():
    ps.append(p.data.numpy())
    
for p, tp in zip(ps, t_ps):
    print(np.allclose(p, tp))

True
True
True
True


In [8]:
y = L(px)
print(y.numpy())

ty = TL(tx)
print(ty.data.numpy())
print(np.allclose(px.numpy(), tx.detach().numpy()))
print(np.allclose(y.numpy(), ty.detach().numpy()))

[[[ 0.67277956  0.08313607 -0.62761104 ... -0.17480263  0.42718208
   -0.5787626 ]
  [ 0.91516656  0.5393416   1.7159258  ...  0.06144593  0.06486575
   -0.03350811]
  [ 0.438351    0.6227843   0.24096036 ...  1.0912522  -0.90929437
   -1.012989  ]
  ...
  [ 0.68631977  0.14240924  0.10763275 ... -0.11513516  0.48065388
    0.04070369]
  [-0.9525228   0.23197874  0.31264272 ...  0.5312439   0.18773697
   -0.8450228 ]
  [ 0.42024016 -0.04561988  0.54541194 ... -0.41933843 -0.00436018
   -0.06663495]]

 [[-0.11638781 -0.33566502 -0.20887226 ...  0.17423287 -0.9195841
   -0.8161046 ]
  [-0.3469874   0.88269687 -0.11887559 ... -0.15566081  0.16357468
   -0.20766167]
  [-0.3847657   0.3984318  -0.06963477 ... -0.00360622  1.2360432
   -0.26811332]
  ...
  [ 0.08230796 -0.46158582  0.54582864 ...  0.15747628 -0.44790155
    0.06020184]
  [-0.8095085   0.43163058 -0.42837143 ...  0.8627463   0.90656304
    0.15847842]
  [-1.485811   -0.18216592 -0.8882585  ...  0.32596245  0.7822631
   -0.646

In [9]:
x = np.random.randn(2, 256)
px = paddle.to_tensor(x, dtype='float32')
tx = torch.tensor(x, dtype=torch.float32)
y = L(px)
print(y.numpy())
ty = TL(tx)
print(ty.data.numpy())
print(np.allclose(px.numpy(), tx.detach().numpy()))
print(np.allclose(y.numpy(), ty.detach().numpy()))
print(np.allclose(y.numpy(), ty.detach().numpy(), atol=1e-5))

[[ 0.04476918  0.554463   -0.3027508  ... -0.49600336  0.3751858
   0.8254095 ]
 [ 0.95594174 -0.29528382 -1.2899452  ...  0.43718258  0.05584608
  -0.06974669]]
[[ 0.04476918  0.5544631  -0.3027507  ... -0.49600336  0.37518573
   0.8254096 ]
 [ 0.95594174 -0.29528376 -1.2899454  ...  0.4371827   0.05584623
  -0.0697467 ]]
True
False
True


In [10]:
print(paddle.version.commit)

5e7e7c9fde8350084abf1898cf52651cfc84b17a


In [11]:
dir(paddle.version)

['__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 'commit',
 'full_version',
 'istaged',
 'major',
 'minor',
 'mkl',
 'patch',
 'rc',
 'show',
 'with_mkl']

In [12]:
print(paddle.version.full_version)

2.1.0


In [13]:
print(paddle.version.show())

commit: 5e7e7c9fde8350084abf1898cf52651cfc84b17a
None


In [14]:
print(torch.__version__)

1.6.0


In [15]:
dir(torch.version)

['__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 '__version__',
 'cuda',
 'debug',
 'git_version',
 'hip']

In [19]:
torch.version.git_version

'b31f58de6fa8bbda5353b3c77d9be4914399724d'

In [21]:
torch.version.cuda

'10.2'